In [102]:
import pandas as pd
import altair as alt

In [103]:
data = pd.read_csv('/Users/siraire/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/Senior Year/Visualization/datasets/energy.csv')
data.head()


,State,Year,Production.Coal,Consumption.Commercial.Coal,Consumption.Commercial.Distillate Fuel Oil,Consumption.Commercial.Geothermal,Consumption.Commercial.Hydropower,Consumption.Commercial.Kerosene,Consumption.Commercial.Petroleum,Consumption.Commercial.Natural Gas,...,Price.Industrial.Coal,Price.Industrial.Distillate Fuel Oil,Price.Industrial.Kerosene,Price.Industrial.Petroleum,Price.Industrial.Natural Gas,Price.Industrial.Other Petroleum Products,Price.Transportation.Coal,Price.Transportation.Distillate Fuel Oil,Price.Transportation.Petroleum,Price.Transportation.Natural Gas
0,Alaska,1960,11263.0,496.0,1562.0,0.0,0.0,0.0,5230.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaska,1961,11497.0,496.0,1646.0,0.0,0.0,0.0,5304.0,122.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alaska,1962,13588.0,593.0,1732.0,0.0,0.0,0.0,5251.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alaska,1963,13307.0,393.0,1868.0,0.0,0.0,0.0,5475.0,1632.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alaska,1964,11622.0,306.0,2020.0,0.0,0.0,0.0,7242.0,1944.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
# throw away useless columns & filter by year
consumption_columns = list(filter(lambda colname : 'Consumption' in colname , data.columns))
data = data[['State', 'Year', *consumption_columns]]
data = data[(data['Year'] >= 2009)]
data = pd.melt(data, id_vars=['State', 'Year'], value_vars=[*consumption_columns])
data['Sector'] = data['variable'].apply(lambda s : s.split('.')[1])
data['Resource'] = data['variable'].apply(lambda s : s.split('.')[2])
data['Consumption'] = data['value']
data = data.drop(['variable', 'value'], axis=1)
data

,State,Year,Sector,Resource,Consumption
0,Alaska,2009,Commercial,Coal,8098.0
1,Alaska,2010,Commercial,Coal,8533.0
2,Alaska,2011,Commercial,Coal,9432.0
3,Alaska,2012,Commercial,Coal,9201.0
4,Alaska,2013,Commercial,Coal,8939.0
...,...,...,...,...,...
21874,Wyoming,2015,Transportation,Natural Gas,13732.0
21875,Wyoming,2016,Transportation,Natural Gas,13515.0
21876,Wyoming,2017,Transportation,Natural Gas,14441.0
21877,Wyoming,2018,Transportation,Natural Gas,16180.0


In [105]:
# create separate dataframes
state_data = data.groupby(['State', 'Resource'])[['Consumption']].mean().reset_index()
state_data

,State,Resource,Consumption
0,Alabama,Coal,85853.818182
1,Alabama,Distillate Fuel Oil,25567.303030
2,Alabama,Geothermal,46.272727
3,Alabama,Hydropower,0.000000
4,Alabama,Kerosene,25.303030
...,...,...,...
556,Wyoming,Other Petroleum Products,14987.454545
557,Wyoming,Petroleum,41411.250000
558,Wyoming,Solar,4.681818
559,Wyoming,Wind,0.000000


In [110]:
# process state_data
total_consumption_per_state = state_data.groupby(['State'])[['Consumption']].sum().reset_index()
def lookupTotalConsumption(state):
    return total_consumption_per_state[total_consumption_per_state['State'] == state]['Consumption']

state_data['Consumption Proportion'] = state_data['Consumption']
for i, row in state_data.iterrows():
    state_data.at[i, 'Consumption Proportion'] =  row['Consumption'] / lookupTotalConsumption(row['State'])

state_data.head()

/var/folders/zt/5pdnryvj4x36h87rs1fzg3pc0000gn/T/ipykernel_84247/3508440505.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  state_data.at[i, 'Consumption Proportion'] =  row['Consumption'] / lookupTotalConsumption(row['State'])


,State,Resource,Consumption,Consumption Proportion
0,Alabama,Coal,85853.818182,0.206746
1,Alabama,Distillate Fuel Oil,25567.303030,0.061569
2,Alabama,Geothermal,46.272727,0.000111
3,Alabama,Hydropower,0.000000,0.000000
4,Alabama,Kerosene,25.303030,0.000061
5,Alabama,Natural Gas,111328.833333,0.268092
6,Alabama,Other Petroleum Products,16545.272727,0.039843
7,Alabama,Petroleum,134640.636364,0.324230
8,Alabama,Solar,16.545455,0.000040
9,Alabama,Wind,0.000000,0.000000


In [135]:
selection = alt.selection(type='multi', fields=['State'])

alt.Chart(state_data).mark_circle().encode(
    x = 'Resource',
    y = 'Consumption Proportion',
    # color = alt.condition(selection, alt.value('magenta'), alt.value('lightblue')),
    size = alt.condition(selection, alt.value(50), alt.value(30)),
    opacity = alt.condition(selection, alt.value(1), alt.value(.05)),
    tooltip = ['State', 'Consumption Proportion']
).properties(width=500).add_selection(selection)

alt.Chart(...)

In [50]:
sector_data = data.groupby(['Sector', 'Resource'])[['Consumption']].mean().reset_index()
sector_data

,Sector,Resource,Consumption
0,Commercial,Coal,787.089127
1,Commercial,Distillate Fuel Oil,6926.456328
2,Commercial,Geothermal,386.368984
3,Commercial,Hydropower,19.705882
4,Commercial,Kerosene,43.342246
5,Commercial,Natural Gas,65431.083779
6,Commercial,Petroleum,14242.055258
7,Commercial,Solar,996.139037
8,Commercial,Wind,17.081996
9,Commercial,Wood,1489.550802


In [136]:
sector_data = data.groupby(['Sector', 'Resource'])[['Consumption']].mean().reset_index()
sector_data

,Sector,Resource,Consumption
0,Commercial,Coal,787.089127
1,Commercial,Distillate Fuel Oil,6926.456328
2,Commercial,Geothermal,386.368984
3,Commercial,Hydropower,19.705882
4,Commercial,Kerosene,43.342246
5,Commercial,Natural Gas,65431.083779
6,Commercial,Petroleum,14242.055258
7,Commercial,Solar,996.139037
8,Commercial,Wind,17.081996
9,Commercial,Wood,1489.550802


In [137]:
# process sector_data
total_consumption_per_sector = sector_data.groupby(['Sector'])[['Consumption']].sum().reset_index()
def lookupTotalConsumption(sector):
    return total_consumption_per_sector[total_consumption_per_sector['Sector'] == sector]['Consumption']

sector_data['Consumption Proportion'] = sector_data['Consumption']
for i, row in sector_data.iterrows():
    sector_data.at[i, 'Consumption Proportion'] =  row['Consumption'] / lookupTotalConsumption(row['Sector'])

sector_data.head()

/var/folders/zt/5pdnryvj4x36h87rs1fzg3pc0000gn/T/ipykernel_84247/3895124625.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sector_data.at[i, 'Consumption Proportion'] =  row['Consumption'] / lookupTotalConsumption(row['Sector'])


,Sector,Resource,Consumption,Consumption Proportion
0,Commercial,Coal,787.089127,0.008713
1,Commercial,Distillate Fuel Oil,6926.456328,0.076672
2,Commercial,Geothermal,386.368984,0.004277
3,Commercial,Hydropower,19.705882,0.000218
4,Commercial,Kerosene,43.342246,0.000480


In [146]:
selection = alt.selection(type='multi', fields=['Sector'])

alt.Chart(sector_data).mark_circle().encode(
    x = 'Resource',
    y = 'Consumption Proportion',
    color = alt.value('#3cb043'),
    # size = alt.value(50),
    size = alt.condition(selection, alt.value(50), alt.value(30)),
    opacity = alt.condition(selection, alt.value(1), alt.value(.3)),
    tooltip = ['Sector', 'Consumption Proportion']
).properties(width=500).add_selection(selection)

alt.Chart(...)